## $\text{Recommender Systems with GNNs} $

GNN을 추천 시스템, 사기 탐지 등과 같은 다양한 분야에 적용하고자 하는 연구가 많이 진행되고 있습니다.

본 tutorial에서는 추천 시스템을 위한 GNN을 다룹니다. 

추천 시스템은 사용자가 선호할만한 제품을 추천하는 시스템입니다. 사용자에게 제품을 추천할 때 활용하는 데이터는 평점 등과 같은 Explicit feedback을 활용하는 경우와 구매 기록, 클릭 등과 같은 implicit feedback을 활용하는 경우로 나눌 수 있습니다. explicit feedback의 경우 모델이 예측한 평점과 실제 사용자가 제품에 부여한 평점 간의 차이를 최소화하는 방향으로 모델을 학습합니다.
$$
\min_{u, v} \sum_{i, j \in \mathcal{D}} (\hat{r}_{i,j} - r_{i, j})^2
$$
추천 시스템에서 가장 대표적인 모델 중 하나인 행렬 분해(Matrix Factorization) 모델의 경우 user vector $u$와 item vector $v$ 간의 내적을 통해 평점을 예측합니다. 

$\hat{r}_{i, j} = u^T_i v_j + \beta_i + \gamma_j $, 이때 $\beta_i$와 $\gamma_j$는 각각 사용자와 제품에 대한 bias를 의미합니다.

In [199]:
# movie lens 데이터를 사용하였습니다. 

import pandas as pd 

train_data = pd.read_csv('data/ua.base', sep='\t', header=None, names=['user_id', 'item_id', 'rating', 'timestamp'])
test_data = pd.read_csv('data/ua.test', sep='\t', header=None, names=['user_id', 'item_id', 'rating', 'timestamp'])
user_data = pd.read_csv('data/u.user', sep='|', header=None, encoding='latin1')
item_data = pd.read_csv('data/u.item', sep='|', header=None, encoding='latin1')

'''
실제 모델을 학습한 후 추천 성능을 측정하기 위해서는 train dataset에 존재하는 사용자 및 제품에 대해서만 사용하여야 합니다. 
따라서, test_data에 train_data에서는 존재하지 않는 사용자와 제품이 존재하는 경우 제거하는 작업을 수행합니다.
'''

test_data = test_data.loc[(test_data.loc[:,'user_id'].isin(train_data.loc[:,'user_id'])) & 
                          (test_data.loc[:,'item_id'].isin(train_data.loc[:,'item_id']))]

In [201]:
user_data.head()

,0,1,2,3,4
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [202]:
item_data.head()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


## $ \text{Heterogeneous Graphs in DGL} $

Heterogeneous Graph는 그래프 내에 노드의 class가 1개가 아닌 그래프를 칭합니다. 

추천 시스템의 경우 사용자 노드와 제품 노드로 되어 있는 Bipartite Graph이기에 Heterogeneous Graph 입니다. 

In [233]:
import dgl 
import torch 

'''
사용자와 제품에 대한 label을 양의 정수 형태로 변환합니다. 
만약 user_id와 label_id가 사용자의 이름 혹은 제품명으로 되어 있는 경우 laebel_encoding을 통해 변환할 수 있습니다.
'''

train_data = train_data.astype({'user_id': 'category', 'item_id': 'category'})
test_data = test_data.astype({'user_id': 'category', 'item_id': 'category'})

# train data과 test data의 사용자와 제품에 대한 label이 동일해야 합니다. 
test_data['user_id'].cat.set_categories(train_data['user_id'].cat.categories, inplace=True)
test_data['item_id'].cat.set_categories(train_data['item_id'].cat.categories, inplace=True)


train_user_ids = torch.LongTensor(train_data['user_id'].cat.codes.values)
train_item_ids = torch.LongTensor(train_data['item_id'].cat.codes.values)
train_ratings = torch.LongTensor(train_data['rating'].values)

test_user_ids = torch.LongTensor(test_data['user_id'].cat.codes.values)
test_item_ids = torch.LongTensor(test_data['item_id'].cat.codes.values)
test_ratings = torch.LongTensor(test_data['rating'].values)


# Build Graph 
# 양방향에 대해서 입력을 해주어야 하기 때문에 user -> item, item -> user 에 대해서 모두 작성해 주어야 합니다. 
# train_user_ids와 train_item_ids는 1D tensor, 1D tensor pair 형태로 입력하셔야 됩니다.
graph = dgl.heterograph({
    ('user', 'watched', 'item'): (train_user_ids, train_item_ids), 
    ('item', 'watched-by', 'user'): (train_item_ids, train_user_ids)
})


c:\Users\EonKim\anaconda3\envs\dgl\lib\site-packages\pandas\core\arrays\categorical.py:2747: FutureWarning: The `inplace` parameter in pandas.Categorical.set_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  res = method(*args, **kwargs)


In [204]:
graph

Graph(num_nodes={'item': 1680, 'user': 943},
      num_edges={('item', 'watched-by', 'user'): 90570, ('user', 'watched', 'item'): 90570},
      metagraph=[('item', 'user', 'watched-by'), ('user', 'item', 'watched')])

* movie genres: one-hot encoding으로 구성되어 있습니다. 
* User age: 사용자의 나이를 10년 단위로 묶어서 categorical variable로 변환합니다.
* User gender: categorical variable
* User occupation(직업): a categorical variable

In [205]:
# user_ids
user_data[0] = user_data[0].astype('category')
user_data[0] = user_data[0].cat.set_categories(train_data['user_id'].cat.categories)
user_data = user_data.dropna(subset=[0]) # train data에 없는 사용자는 제거합니다. 
user_data[0] = user_data[0].cat.codes
user_data = user_data.sort_values(0)

item_data[0] = item_data[0].astype('category')
item_data[0] = item_data[0].cat.set_categories(train_data['item_id'].cat.categories)
item_data = item_data.dropna(subset=[0]) # train data에 없는 제품은 제거합니다. 
item_data[0] = item_data[0].cat.codes
item_data = item_data.sort_values(0)

# gender
user_data[2] = user_data[2].astype('category')

# occupation
user_data[3] = user_data[3].astype('category')

# ??
user_data[4] = user_data[4].astype('category')



user_age = user_data[1].values // 10
num_user_age_bins = user_age.max() + 1     # count the number of user age bins
user_gender = user_data[2].cat.codes.values
num_user_genders = len(user_data[2].cat.categories)
user_occupation = user_data[3].cat.codes.values
num_user_occupations = len(user_data[3].cat.categories)

item_genres = item_data[range(5, 24)].values # one-hot encoding 
num_item_genres = item_genres.shape[1]

In [206]:
graph.nodes['user'].data['age'] = torch.LongTensor(user_age)
graph.nodes['user'].data['gender'] = torch.LongTensor(user_gender)
graph.nodes['user'].data['occupation'] = torch.LongTensor(user_occupation)

graph.nodes['item'].data['genres'] = torch.FloatTensor(item_genres)

# weight edges
graph.edges['watched'].data['rating'] = torch.LongTensor(train_ratings)
graph.edges['watched-by'].data['rating'] = torch.LongTensor(train_ratings)

In [207]:
from torch.utils.data import TensorDataset, DataLoader

# TensorDataset의 경우 Dataset과는 달리 Tensor만 입력으로 받을 수 있습니다.
train_dataset = TensorDataset(train_user_ids, train_item_ids, train_ratings)
test_dataset = TensorDataset(test_user_ids, test_item_ids, test_ratings)

## $\text{Define Minibatch \& Neighbor Sampler} $

multi-layer GNN을 사용하기 위해서는 먼저 Minibatch sampler를 정의해야 합니다. 

추천 시스템에서는 사용자와 제품 간의 종속성(Dependency)이 존재하기 때문에 이를 고려하여 계산하기 위함입니다. 


**neighbor sampler**
1. batch size 만큼의 pair_graph를 추출합니다. (heterograph를 생성)
2. compact_graph로 변환합니다. parir_graph로 sampling된 node 중 아무 노드와 연결되어 있지 않는 노드를 제거하는 작업 입니다. 
3. block을 생성합니다.

**construct_blocks**
1. graph의 edge(link) 정보를 불러옵니다. 
2. 모델 학습을 위해 edge의 정보를 제거합니다. (모델이 사용자와 제품이 연결되어 있는 정보를 알고 있으면 학습하는 것에 의미가 없기 때문.)
3. edge 정보를 제거한 graph를 block에 저장합니다. 

In [234]:
class MinibatchSampler(object):
    def __init__(self, graph, num_layers):
        self.graph = graph
        self.num_layers = num_layers
        
    def sample(self, batch):
        users, items, ratings = zip(*batch)
        users = torch.stack(users)
        items = torch.stack(items)
        ratings = torch.stack(ratings)
        
        # Create a pair graph (Step 1)
        pair_graph = dgl.heterograph(
            {('user', 'watched', 'item'): (users, items)},
            num_nodes_dict={'user': self.graph.number_of_nodes('user'), 'item': self.graph.number_of_nodes('item')})
        
        # Compact the graph (Step 2)
        # compact_graph는 isolated node를 찾아서 제거하는 역할을 합니다. 구체적으로 node의 degree, out-degree가 0인 node를 제거합니다.
        pair_graph = dgl.compact_graphs(pair_graph)
        
        # Assign ratings to the graph
        pair_graph.edata['rating'] = ratings
        
        # Construct blocks (Step 3)
        # 분산처리를 하기 위한 전처리 과정입니다.
        # NID/EID는 subgraph의 node와 edge들이 reshuffle 후의 전체 graph의 새로운 node/edge ID를 저장합니다. 
        # 학습이 실행되는 동안, 새로운 node/edge ID만을 사용하는 것입니다. (node와 edge를 구별하기 위함입니다.)
        seeds = {'user': pair_graph.nodes['user'].data[dgl.NID],
                 'item': pair_graph.nodes['item'].data[dgl.NID]}
        blocks = self.construct_blocks(seeds, (users, items))
        
        for feature_name in self.graph.nodes['user'].data.keys():
            blocks[0].srcnodes['user'].data[feature_name] = \
                self.graph.nodes['user'].data[feature_name][blocks[0].srcnodes['user'].data[dgl.NID]]
                
        for feature_name in self.graph.nodes['item'].data.keys():
            blocks[0].srcnodes['item'].data[feature_name] = \
                self.graph.nodes['item'].data[feature_name][blocks[0].srcnodes['item'].data[dgl.NID]]

        return pair_graph, blocks
    
    def construct_blocks(self, seeds, user_item_pairs_to_remove):
        blocks = []
        users, items = user_item_pairs_to_remove
        for i in range(self.num_layers):
            sampled_graph = dgl.in_subgraph(self.graph, seeds) # seed로 지정된 subgraph를 반환합니다. 
            
            sampled_eids = sampled_graph.edges['watched'].data[dgl.EID]
            sampled_eids_rev = sampled_graph.edges['watched-by'].data[dgl.EID]
            
            # rating을 예측하는 것은 edge를 예측하는 것과 같으며, 
            # sub graph의 edge를 예측할 때 모델이 연결되어 있다는 정보를 알지 못하도록 remove 합니다.
            # 모델이 연결되어 있다는 정보를 알고 있다면, 예측의 의미가 없기 때문입니다.
            _, _, edges_to_remove = sampled_graph.edge_ids(users, items, etype='watched', return_uv=True)
            _, _, edges_to_remove_rev = sampled_graph.edge_ids(items, users, etype='watched-by', return_uv=True)
            
            sampled_with_edges_removed = sampled_graph 
            
            if len(edges_to_remove) > 0:
                sampled_with_edges_removed = dgl.remove_edges(
                    sampled_with_edges_removed, edges_to_remove, 'watched'
                )
                sampled_eids = sampled_eids[sampled_with_edges_removed.edges['watched'].data[dgl.EID]]
            
            if len(edges_to_remove_rev) > 0:
                sampled_with_edges_removed = dgl.remove_edges(
                    sampled_with_edges_removed, edges_to_remove_rev, 'watched-by'
                )
                sampled_eids_rev = sampled_eids_rev[sampled_with_edges_removed.edges['watched-by'].data[dgl.EID]]
                
            # Create a block from the sampled graph.
            block = dgl.to_block(sampled_with_edges_removed, seeds)
            blocks.insert(0, block)
            seeds = {'user':block.srcnodes['user'].data[dgl.NID], 
                     'item':block.srcnodes['item'].data[dgl.NID]}
            
            # copy the ratings to the edges of sampled block
            block.edges['watched'].data['rating'] = \
                self.graph.edges['watched'].data['rating'][sampled_eids]
            
            block.edges['watched-by'].data['rating'] = \
                self.graph.edges['watched-by'].data['rating'][sampled_eids_rev]
        
        return blocks 

## $\text{Define Model}$

본 tutorial에서 사용하는 $\text{Graph Convolutional Matrix Completion} $(GCMC)은 매우 간단한 버전 입니다. 

GCMC는 각 노드의 representation을 계산하여 이웃에게 message를 보냅니다. 

각각의 노드는 전달받은 message를 모으고, 평균을 취해 node representation으로 사용합니다. 

$ \ $

$l^{\text{th}}-layer$의 representation을 update하는 경우:

1. node $i$와 interaction이 존재하는 모든 이웃을 찾습니다.

2. 그 후, user $i$가 item $j$에 남긴 rating $r_{ij}$를 확인하고 linear projection을 통해 message를 전달합니다. 
$$
m^l_{j \rightarrow i} \leftarrow W^l_{r_{ij}}v^{l-1}_j
$$

3. node $i$와 전달받은 message를 합계하여 $u^l_i$를 update 합니다. 
$$
u^l_i \leftarrow \text{ReLU} ( W^l [\sum(m^l_{j \rightarrow i}); u^{l-1}_i])
$$

$ \ $

실제 모델을 학습할 때 edge 즉, rating을 제거했습니다. 따라서, 모델은 $W^l_{r_ij}, W^l$을 학습하며 최적화 합니다. 

![image_1](asset/figure_1.JPG)

In [235]:
import torch.nn as nn 
import torch.nn.functional as F 
import dgl.function as fn 
import dgl.nn as dglnn 

In [236]:
class GCMCConv(nn.Module):
    def __init__(self, hidden_dims, num_ratings):
        super().__init__()
        
        # rating은 1부터 시작하기 때문에 + 1을 합니다. 
        self.W_r = nn.Parameter(torch.randn(num_ratings+1, hidden_dims, hidden_dims))
        self.W = nn.Linear(hidden_dims*2, hidden_dims)
        
    def compute_message(self, W, edges):
        W_r = W[edges.data['rating']]
        h = edges.src['h']
        m = (W_r @ h.unsqueeze(-1)).squeeze(2)
        return m 
    
    def forward(self, graph, node_features):
        with graph.local_scope():
            src_features, dst_features = node_features
            graph.srcdata['h'] = src_features 
            graph.dstdata['h'] = dst_features 
            
            # Compute messages 
            graph.apply_edges(lambda edges: {'m': self.compute_message(self.W_r, edges)})
            
            # Aggregate messages 
            graph.update_all(fn.copy_e('m', ','), fn.mean('m', 'h_neigh'))
            
            # Update the representations of output users and items 
            result = F.relu(self.W(torch.cat([graph.dstdata['h'], graph.dstdata['h_neigh']], 1)))
            return result

In [237]:
class GCMCLayer(nn.Module):
    def __init__(self, hidden_dims, num_ratings):
        super().__init__()
        
        self.heteroconv = dglnn.HeteroGraphConv(
            {'watched': GCMCConv(hidden_dims, num_ratings), 'watched-by': GCMCConv(hidden_dims, num_ratings)}, 
            aggregate='sum'
        )
    
    def forward(self, block, input_user_features, input_item_features):
        with block.local_scope():
            h_user = input_user_features 
            h_item = input_item_features 
            
            src_features = {'user':h_user, 'item':h_item}
            
            dst_features = {'user':h_user[:block.number_of_dst_nodes('user')], 'item': h_item[:block.number_of_dst_nodes('item')]}
            
            result = self.heteroconv(block, (src_features, dst_features))
            return result['user'], result['item']

In [238]:
class GCMCRating(nn.Module):
    def __init__(self, num_users, num_items, hidden_dims, num_ratings, num_layers):
        super().__init__()
        
        self.user_embeddings = nn.Embedding(num_users, hidden_dims)
        self.item_embeddings = nn.Embedding(num_items, hidden_dims)
        
        self.U_age = nn.Embedding(num_user_age_bins, hidden_dims)
        self.U_gender = nn.Embedding(num_user_genders, hidden_dims)
        self.U_occupation = nn.Embedding(num_user_occupations, hidden_dims)
        self.U_genres = nn.Linear(num_item_genres, hidden_dims)
        
        self.layers = nn.ModuleList([
            GCMCLayer(hidden_dims, num_ratings) for _ in range(num_layers)
        ])
        
        self.W = nn.Linear(hidden_dims, hidden_dims)
        self.V = nn.Linear(hidden_dims, hidden_dims)
        
    def forward(self, blocks):
        user_embeddings = self.user_embeddings(blocks[0].srcnodes['user'].data[dgl.NID])
        item_embeddings = self.item_embeddings(blocks[0].srcnodes['item'].data[dgl.NID])
        
        # user_embedding에 사용할 user_feature를 추가하는 작업을 수행합니다. 
        user_embeddings += self.U_age(blocks[0].srcnodes['user'].data['age'])
        user_embeddings += self.U_gender(blocks[0].srcnodes['user'].data['gender'])
        user_embeddings += self.U_occupation(blocks[0].srcnodes['user'].data['occupation'])
        item_embeddings += self.U_genres(blocks[0].srcnodes['item'].data['genres'])
        
        for block, layer in zip(blocks, self.layers):
            user_embeddings, item_embeddings = layer(block, user_embeddings, item_embeddings)
            
        user_embeddings = self.W(user_embeddings)
        item_embeddings = self.V(item_embeddings)
        
        return user_embeddings, item_embeddings 
    
    def compute_score(self, pair_graph, user_embeddings, item_embeddings):
        with pair_graph.local_scope():
            pair_graph.nodes['user'].data['h'] = user_embeddings 
            pair_graph.nodes['item'].data['h'] = item_embeddings 
            pair_graph.apply_edges(fn.u_dot_v('h', 'h', 'r'))
            
            return pair_graph.edata['r']

## $\text{Define Evaluation Metric} $

In [239]:
def rmse(pred, label):
    return ((pred-label)^2).mean().sqrt()

## $\text{Train Loop} $

In [ ]:
import tqdm 

NUM_LAYERS = 1 
BATCH_SIZE = 500 
NUM_EPOCHS = 50 
HIDDEN_DIMS = 8

sampler = MinibatchSampler(graph, NUM_LAYERS)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, collate_fn=sampler.sample, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, collate_fn=sampler.sample, shuffle=False)

model = GCMCRating(graph.number_of_nodes('user'), graph.number_of_nodes('item'), HIDDEN_DIMS, 5, NUM_LAYERS)
optimizer = torch.optim.Adam(model.parameters())

for _ in range(NUM_EPOCHS):
    model.train()
    with tqdm.tqdm(train_dataloader) as t:
        for pair_graph, blocks in t: 
            user_emb, item_emb = model(blocks)
            prediction = model.compute_score(pair_graph, user_emb, item_emb)
            loss = ((prediction - pair_graph.edata['rating']) ** 2).mean()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            t.set_postfix({'loss': '%.4f' % loss.item()}, refresh=False)
    
    model.eval()
    with tqdm.tqdm(test_dataloader) as t:
        with torch.no_grad():
            predictions = []
            ratings = []
            for pair_graph, blocks in t:
                user_emb, item_emb = model(blocks)
                prediction = model.compute_score(pair_graph, user_emb, item_emb)
                predictions.append(prediction)
                ratings.append(pair_graph.edata['rating'])
            
            predictions = torch.cat(predictions, dim=0)
            ratings = torch.cat(ratings, dim=0)
        
        print('RMSE', rmse(predictions, ratings).item())

## $\text{Option Implicit Feedback} $

In [ ]:
class LinkPredictionMinibatchSampler(MinibatchSampler):
    def __init__(self, graph, num_layers):
        self.graph = graph
        self.num_layers = num_layers
        
    def sample(self, batch):
        # Convert the list of user-item-rating triplets into a pairs of users and items
        users, items, _ = zip(*batch)
        users = torch.stack(users)
        items = torch.stack(items)
        # negative sampling
        neg_items = torch.randint(0, self.graph.number_of_nodes('item'), (len(users),))
        
        # Create a pair graph for positive examples and negative examples (Step 1)
        pos_pair_graph = dgl.heterograph(
            {('user', 'watched', 'item'): (users, items)},
            num_nodes_dict={'user': self.graph.number_of_nodes('user'), 'item': self.graph.number_of_nodes('item')})
        neg_pair_graph = dgl.heterograph(
            {('user', 'watched', 'item'): (users, neg_items)},
            num_nodes_dict={'user': self.graph.number_of_nodes('user'), 'item': self.graph.number_of_nodes('item')})
        
        # Compact the graph (Step 2)
        pos_pair_graph, neg_pair_graph = dgl.compact_graphs([pos_pair_graph, neg_pair_graph])
        
        # Construct blocks (Step 3)
        # Note that pos_pair_graph and neg_pair_graph have the same set of users and items, so we only need
        # to check one of them to get the seed nodes.
        seeds = {'user': pos_pair_graph.nodes['user'].data[dgl.NID],
                 'item': pos_pair_graph.nodes['item'].data[dgl.NID]}
        # Note that here we would also remove edges connecting between users and both the corresponding positive
        # and negative items appearing in the minibatch.
        blocks = self.construct_blocks(seeds, (torch.cat([users, users]), torch.cat([items, neg_items])))
        
        # Copy node features from original graph to the sampled block.
        # Note that for our model we only need to copy the features to the source side of the first block.
        for feature_name in self.graph.nodes['user'].data.keys():
            blocks[0].srcnodes['user'].data[feature_name] = \
                self.graph.nodes['user'].data[feature_name][blocks[0].srcnodes['user'].data[dgl.NID]]
        for feature_name in self.graph.nodes['item'].data.keys():
            blocks[0].srcnodes['item'].data[feature_name] = \
                self.graph.nodes['item'].data[feature_name][blocks[0].srcnodes['item'].data[dgl.NID]]
            
        return pos_pair_graph, neg_pair_graph, blocks